In [1]:
import datetime
import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

#import emotion_model
import dwdii_bc_model_helper as bc
import bc_models as models

random.seed(20275)

Using Theano backend.


In [2]:
print "device:", theano.config.device
print "floatX:",  theano.config.floatX
print "mode:", theano.config.mode
print "openmp:", theano.config.openmp
print "allow_gc:", theano.config.allow_gc

device: cpu
floatX: float32
mode: FAST_RUN
openmp: True
allow_gc: False


In [3]:
imagePath = "/root/bc_data/ddsm-png.25"
dataPath = "/root/bc_data/ddsm-png.25/Ddsm_png.csv"
imgResize = (150, 150)

In [4]:
os.listdir('/root/bc_data')

['ddsm', 'ddsm-hog', 'ddsm-png.25', 'ddsm-sm', 'Ddsm.csv', 'mias']

# Load Training and Test Data

In this section, the training/validation data is loaded. The load_data function pre-balances the data set by removing images from over-represented emotion classes.

In [5]:
metaData, meta2 = bc.load_training_metadata(dataPath, verbose=True)

Before Balancing
----------------
benign 3164
malignant 1096

After Balancing
----------------
benign 3164
malignant 1096


In [7]:
meta2[meta2.keys()[0]]

'0'

In [8]:
# TODO: Actually load some representative data for model experimentation
maxData = 1000
X_data, Y_data = bc.load_data(dataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_data.shape
print Y_data.shape

Before Balancing
----------------
benign 3164
malignant 1096

After Balancing
----------------
benign 3164
malignant 1096
0.000000: A_0152_1.RIGHT_MLO.LJPEG.png
0.050000: C_0128_1.RIGHT_MLO.LJPEG.png
0.100000: B_3511_1.RIGHT_CC.LJPEG.png
0.150000: A_0362_1.LEFT_CC.LJPEG.png
0.200000: C_0022_1.RIGHT_MLO.LJPEG.png
0.250000: B_3663_1.LEFT_MLO.LJPEG.png
0.300000: A_1084_1.LEFT_CC.LJPEG.png
0.350000: A_0235_1.RIGHT_MLO.LJPEG.png
0.400000: A_0034_1.LEFT_CC.LJPEG.png
0.450000: C_0027_1.LEFT_CC.LJPEG.png
0.500000: A_1044_1.RIGHT_MLO.LJPEG.png
0.550000: C_0230_1.LEFT_CC.LJPEG.png
0.600000: B_3672_1.RIGHT_MLO.LJPEG.png
0.650000: A_0091_1.LEFT_MLO.LJPEG.png
0.700000: A_0508_1.LEFT_MLO.LJPEG.png
0.750000: B_3019_1.RIGHT_CC.LJPEG.png
0.800000: A_0490_1.RIGHT_CC.LJPEG.png
0.850000: A_0249_1.LEFT_CC.LJPEG.png
0.900000: A_1104_1.RIGHT_CC.LJPEG.png
0.950000: A_0710_1.RIGHT_CC.LJPEG.png
(1000, 150, 150)
(1000, 1)


## Split Training/Test Sets
The following code segment splits the data into training and test data sets. Currently this is a standard 80/20 split for training and test respectively after performing a random shuffle using the unison_shuffled_copies help method.

In [9]:
skippedTransforms = True
if skippedTransforms:
    X_data2 = X_data
    Y_data2 = Y_data

In [10]:
# Split the data into Training and Test sets
trainNdx = int(X_data2.shape[0] * .8)
print trainNdx

X_train, X_test = np.split(X_data2, [trainNdx])
Y_train, Y_test = np.split(Y_data2, [trainNdx])
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

800
(800, 150, 150)
(200, 150, 150)
(800, 1)
(200, 1)


## Define and Load Trained Model

In [11]:
# Load the bc array for our count in the model definition
bcTypes = bc.bcNumerics()
print bcTypes
print len(bcTypes)

{'benign': 0, 'malignant': 1}
2


In [12]:
# Construct the model using our help function
model = models.bc_model_v0(len(bcTypes), verbose=True, 
                                        input_shape=(1,X_train.shape[1],X_train.shape[2]))

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 32, 143, 143)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 67, 67)  25632       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

## Training the Model

The following code segment trains the model using the run_network helper function. 

In [13]:
loadWeights = False
if loadWeights:
    model.load_weights("dwdii-bc-150-v0-Cloud.hdf5")

In [14]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, X_train.shape[1],X_train.shape[2])
trainX = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],X_train.shape[2])

In [ ]:
print "Training start: " + str(datetime.datetime.now())
m, h = models.run_network([trainX, testX, Y_train, Y_test], model, batch=50, epochs=30, verbosity=1)

Training start: 2017-03-20 03:12:14.703137
(800, 2)
(200, 2)
Training model...
Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 187s - loss: 0.7914 - acc: 0.7275 - val_loss: 0.5522 - val_acc: 0.7250
Epoch 2/30
800/800 [==============================] - 205s - loss: 0.5123 - acc: 0.7537 - val_loss: 0.5719 - val_acc: 0.7250
Epoch 3/30
800/800 [==============================] - 190s - loss: 0.5097 - acc: 0.7562 - val_loss: 0.5951 - val_acc: 0.7200
Epoch 4/30
800/800 [==============================] - 217s - loss: 0.5197 - acc: 0.7550 - val_loss: 0.6463 - val_acc: 0.7250
Epoch 5/30
800/800 [==============================] - 188s - loss: 0.4833 - acc: 0.7537 - val_loss: 0.6101 - val_acc: 0.5350
Epoch 6/30
800/800 [==============================] - 184s - loss: 0.5127 - acc: 0.7325 - val_loss: 0.6077 - val_acc: 0.7200
Epoch 7/30
800/800 [==============================] - 185s - loss: 0.4787 - acc: 0.7338 - val_loss: 0.5629 - val_acc: 0.7150


In [ ]:
model.save_weights("dwdii-bc-150-v0-Cloud.hdf5", overwrite=True)